In [ ]:
import os
import h5py
import json
import pandas as pd

In [ ]:
root = "/home/jluesch/Documents/data/plankton"

In [ ]:
files = os.listdir(root)

In [ ]:
acc_ids = []
acc_names = []

all_files_entries = dict()
class_names_dict = dict()
for hdf5_file in files:
    print(hdf5_file, end=" ")
    all_files_entries[hdf5_file] = []
    class_ids = []
    class_names = []
    hdf5_file_path = os.path.join(root, hdf5_file)
    file = h5py.File(hdf5_file_path, "r")

    # Read the JSON string from the 'file_index' dataset
    file_index_json = file["file_index"][()]
    file_index = json.loads(file_index_json)

    # Add the HDF5 file name to each entry and accumulate the file entries
    for entry in file_index["files"]:
        entry["hdf5_file"] = hdf5_file_path  # Add the HDF5 file name to the entry
        all_files_entries[hdf5_file].append(entry)
        class_id = entry["class_id"]
        class_str = entry["class_str"]
        if class_id not in class_ids:
            class_ids.append(class_id)
            class_names.append(class_str)

        if class_id not in acc_ids:
            acc_ids.append(class_id)
            acc_names.append(class_str)
    class_names_dict[hdf5_file] = class_names

    print(len(class_ids))

In [ ]:
class_count = dict()
for entry in entries[:10]:
    print(entry)

In [ ]:
len(acc_ids)

In [ ]:
class_names_dict.keys()

In [ ]:
all_names = set()
for v in class_names_dict.values():
    all_names = all_names.union(set(v))

In [ ]:
print(len(all_names))

In [ ]:
for k, v in class_names_dict.items():
    print(k, " missing :", all_names.difference(set(v)))